This notebook will be used to upload, search and plot using the HUGS Cloud platform

In [ ]:
import os
import sys
from datetime import datetime
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve

from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Axis, Lines, Figure
from random import randint    

from numpy import random as np_random
import numpy as np

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Login

Login to the platform

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
user = User(username="gareth", identity_url=F"{base_url}/identity")
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/f4.2a.1e.64
(please check that this page displays the message 'white cows stand quickly')


Check we've logged in successfully

In [ ]:
user.wait_for_login()

True

## Upload

First we want to upload the data to the cloud platform

In [ ]:
# Helper function for test file paths
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

Get a process object so we can send files to the platform

In [ ]:
processing = Process(service_url=base_url)

For the time being clear Datasources

In [ ]:
hugs = Service(service_url="%s/hugs" % base_url)
hugs.call_function(function="clear_datasources", args={})

Upload some data

In [ ]:
bsd_file = get_CRDS_path("bsd.picarro.1minute.248m.dat")
hfd_file = get_CRDS_path("hfd.picarro.1minute.100m_min.dat")

result_bsd = processing.process_files(user=user, files=bsd_file, data_type="CRDS")
result_hfd = processing.process_files(user=user, files=hfd_file, data_type="CRDS")

In [ ]:
print(result_bsd, "\n\n", result_hfd)

{'bsd.picarro.1minute.248m.dat': ['b2df114a-f031-4eb0-bfa0-ce0064f2a529', '45e422a6-f9d5-41cf-8b79-c5bf28f50b09', '05079df2-4bf7-49c6-8d3a-2e1e60e4d296']} 

 {'hfd.picarro.1minute.100m_min.dat': ['614b70c0-b129-4460-a9ab-8eb0b0f65af2', '65559e79-3446-4ec3-8dad-d50f59acf6d8', 'b4af6223-bf6a-4dc2-b1bb-7d122d666d9f']}


Get a search object

In [ ]:
search = Search(service_url=base_url)

In [ ]:
start = datetime(1970, 1,1)
end = datetime.now()

search_terms = ["co", "co2"]
search_locations = ["bsd", "hfd"]
data_type = "CRDS"

search_results = search.search(search_terms=search_terms, locations=search_locations, 
                               data_type=data_type, start_datetime=start, end_datetime=end)


['co', 'co2'] ['bsd', 'hfd']


Get the keys we want 

In [ ]:
print(list(search_results.keys()))

['bsd_co', 'hfd_co', 'bsd_co2', 'hfd_co2']


The items stored at each key are the keys for the data

In [ ]:
search_results["bsd_co"]

['data/uuid/05079df2-4bf7-49c6-8d3a-2e1e60e4d296/2014-01-30T10:52:30_2014-01-30T14:20:30']

Now create a list of the terms we want to download

In [ ]:
to_download = ["bsd_co", "hfd_co"]

Now we can (for now) pull this data from the object store for use

In [ ]:
retrieve = Retrieve(service_url=base_url)

In [ ]:
# download_keys = {key: date_keys[key]["keys"] for key in selected_data}
# download_keys = {"keys": search_results["bsd_co"], "return_type": "json"}
# download_keys

{'keys': ['data/uuid/05079df2-4bf7-49c6-8d3a-2e1e60e4d296/2014-01-30T10:52:30_2014-01-30T14:20:30'],
 'return_type': 'json'}

In [ ]:
data = retrieve.retrieve(keys=download_keys)

IndexError: Error calling 'retrieve' on 'https://hugs.acquire-aaai.com/t/hugs': list index out of range